# Imports

In [1]:
from bonsai.nas import Bonsai
from bonsai.helpers import mem_stats

%load_ext autoreload
%autoreload 2

mem_stats()

' 512.00B  '

# Define Hypers

In [2]:
hypers = {
    'gpu_space':8.5,
    'dataset':{'name':'CIFAR10', 'classes':10},
    'batch_size':64,
    'scale':36,
    'nodes':4,
    'patterns':[['n','na'], ['r']],
    'reduction_target':2,
    'lr_schedule': {'lr_max': .025, 'T': 600},
    'drop_prob':.3,
    'nas_schedule': {'prune_interval':4, 'cycle_len':8},
    'prune_rate':{'edge':.01, 'input':.01}
}

# Run

In [ ]:
bonsai = Bonsai(hypers)
bonsai.train()

== Determining compression ratios ==
Search Range: 0.14->1.00
Comp Ratios:
sizes={
    1:0.8928571428571428,
    2:0.46428571428571425,
    3:0.5178571428571428,
    4:0.4107142857142857,
}
start_size=1
Effective Scale: 103.50
=========================== NETWORK ===========================
================= Mindanao Samuelson Riviera ==================
                     :     Dim      :    Params    :   Comp   
Initializer          :              :     180      :          
Cell 0  (Normal)     :   36 x 32    :   167,859    :  100.0%  
Cell 1  (Normal)     :   36 x 32    :   167,860    :  100.0%  
 ↳ Classifier        :              :   368,650    :          
Total                :              :   704,549    :  100.0%  

=== 1 Patterns. Target Comp: 0.89, Aim: 0.80
=== Training Mindanao Samuelson Riviera ===
Starting at 2020-02-22 11:00:10.888694
02/22/2020 11:00 AM
Init:   24.00MiB
0:    3.73GiB
1:    7.23GiB
GP:    7.23GiB
Classifier:    7.23GiB
Train Epoch: 0   [12512 /50000  (10

In [ ]:
bonsai.random_search(1)

In [ ]:
bonsai.random_search(2)

In [ ]:
bonsai.random_search(3)